In [6]:
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from utils import *

In [2]:
class H36M(Dataset):

    def __init__(self, label_path = '/Users/byjiang/Code/Data/H36M_pose.npy',istrain=True):
        super().__init__()
        label = np.load(label_path,allow_pickle=True).item()
        self.istrain = istrain
        
        self.cameras = label['subject_cameras']
        self.db = label['train'] if self.istrain else label['test']

    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        """
        pose/3d : (17,3)
        R : (4,3,3)
        t : (4,3,1)
        K : (4,3,3)
        pose/2d : (4,17,2)
        confi : (4,17)
        B,V,J,D,1
        B,4,17,3,1
        """
        frame = self.db[index]
        camera = self.cameras[frame['subject_index']]
        R, t = camera['R'].copy(), camera['t'].copy()

        pose_3d = (R[0,None] @ frame['pose/3d'][...,None] + t[0,None]).squeeze(-1)
        R[1:] = R[1:] @ np.linalg.inv(R[0])[None]
        t[1:] = t[1:] - R[1:] @ t[0,None]
        R[0] = np.eye(3)
        t[0] = np.zeros((3,1))

        # X_2d = R X_3d + t
        pose_2d = homo_to_eulid( (R[:,None] @ pose_3d[None,...,None] + t[:,None]).squeeze(-1))
        confi = np.ones(pose_2d.shape[:-1])
        return pose_3d, pose_2d, confi, R, t
        # return {
        #     'pose/3d': frame['pose/3d'],
        #     'pose/2d':  homo_to_eulid( (camera['R'][:,None] @ frame['pose/3d'][...,None] + camera['t'][:,None]).squeeze(-1)),
        #     'R': camera['R'],
        #     't': camera['t']
        #     # 'K': camera['K']
        # }


In [3]:
h36m = H36M()
h36mloader = DataLoader(h36m, batch_size = 2, shuffle = True)

In [14]:

b = torch.randn((10,2))
c = torch.randn((10,2))
cv2.findEssentialMat(b,c, focal=1.0, pp=(0., 0.),
                                        method=cv2.RANSAC, prob=0.999, threshold=0.0003)

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'findEssentialMat'
> Overload resolution failed:
>  - findEssentialMat() missing required argument 'cameraMatrix' (pos 3)
>  - findEssentialMat() missing required argument 'cameraMatrix' (pos 3)
>  - points1 is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'points1'
>  - findEssentialMat() missing required argument 'cameraMatrix1' (pos 3)
>  - findEssentialMat() missing required argument 'cameraMatrix1' (pos 3)
>  - findEssentialMat() missing required argument 'cameraMatrix1' (pos 3)
>  - findEssentialMat() missing required argument 'cameraMatrix1' (pos 3)


In [ ]:
class CalibrationBatch():
    def __init__(self, points2d, confi2d):
        """
        points2d : (B,V,J,2)
        confi2d : (B,V,J)
        points3d : (B,J,3)
        confi3d : (B,J)
        R : (B,V,3,3)
        t : (B,V,3,1)
        """
        self.n_batch,self.n_view,self.n_joint = points2d.shape[:3]
        self.points2d = points2d
        self.confi2d = confi2d
        self.points3d = torch.zeros((self.n_batch,self.n_joint,3))
        self.confi3d = torch.zeros((self.n_batch,self.n_joint))
        self.R = torch.zeros((self.n_batch,self.n_view,3,3))
        self.t = torch.zeros((self.n_batch,self.n_view,3,1))

    def weighted_triangulation(self, points2d, confi2d, R ,t):
        """
        Args:
            points2d : (V',J,2)
            confi2d : (V',J)
            R : (V',3,3)
            t : (V',3,1)
        Returns:
            points3d : (J,3)
            confi3d : (J)
        """
        n_view_filter= points2d.shape[0]
        points3d = torch.zeros((self.n_joint, 3))
        confi3d = torch.zeros((self.n_joint))
        for j in range(self.n_joint):
            A = []
            for i in range(n_view_filter):
                if confi2d[i,j] > 0.5:
                    P = torch.cat([R[i],t[i]],dim=1)
                    P3T = P[2]
                    A.append(confi2d[i,j] * (points2d[i,j,0]*P3T - P[0]))
                    A.append(confi2d[i,j] * (points2d[i,j,1]*P3T - P[1]))
            A = torch.stack(A)
            if A.shape[0] >= 4:
                u, s, vh = torch.linalg.svd(A)
                error = s[-1]
                X = vh[len(s) - 1]
                points3d[:,j] = X[:3] / X[3]
                confi3d[j] = np.exp(-torch.abs(error))
            else:
                points3d[:,j] = torch.tensor([0.0,0.0,0.0])
                confi3d[j] = 0

        return points3d, confi3d

    def pnp(self,batch_id):
        for i in range(self.n_view):
            mask = torch.logical_and(self.confi2d[batch_id,i]>0.8,self.confi3d[batch_id]>0.8)
            p2d = self.points2d[batch_id,i,mask].numpy()
            p3d = self.points3d[batch_id,mask].numpy()
            ret, rvec, tvec = cv2.solvePnP(p3d,p2d, np.eye(3), np.zeros(5))
            R, _ = cv2.Rodrigues(rvec)
            self.R[batch_id,i] = torch.tensor(R)
            self.R[batch_id,i] = torch.tensor(tvec)


    def eight_point(self):
        for batch_id in range(self.n_batch):
            mask = torch.logical_and(self.confi2d[batch_id,0]>0.8, self.confi2d[batch_id,1]>0.8)
            
            p0 = self.points2d[batch_id,0,mask].numpy()
            p1 = self.points2d[batch_id,1,mask].numpy()
            # p0,p1 (N,2)
            E, mask = cv2.findEssentialMat(p0, p1, focal=1.0, pp=(0., 0.),
                                            method=cv2.RANSAC, prob=0.999, threshold=0.0003)
            p0_inliers = p0[mask.ravel() == 1]
            p1_inliers = p0[mask.ravel() == 1]
            point, R, t,mask  = cv2.recoverPose(E, p0_inliers, p1_inliers)
            self.R[batch_id,0],self.t[batch_id,0] = torch.eye(3), torch.zeros(3,1)
            self.R[batch_id,1],self.t[batch_id,1] = R,t

            self.points3d[batch_id], self.confi3d[batch_id] = self.weighted_triangulation(
                self.points2d[batch_id,:2],self.confi2d[batch_id,:2],self.R[batch_id,:2],self.t[batch_id,:2]
            )
            
            self.pnp(batch_id)

            self.points3d[batch_id], self.confi3d[batch_id] = self.weighted_triangulation(
                self.points2d[batch_id],self.confi2d[batch_id],self.R[batch_id],self.t[batch_id]
            )

In [7]:
calibr = Calibration(4,)

TypeError: __init__() missing 2 required positional arguments: 'n_view' and 'n_joint'

In [4]:
for step, (pose_3d, pose_2d, confi, R, t) in enumerate(h36mloader):
    print(pose_2d.shape,pose_3d.shape, confi.shape,R.shape,t.shape)
    break


torch.Size([2, 4, 17, 2]) torch.Size([2, 17, 3]) torch.Size([2, 4, 17]) torch.Size([2, 4, 3, 3]) torch.Size([2, 4, 3, 1])


In [5]:
cal_mpjpe(pose_3d,pose_2d,R,t)

tensor(0.)